In [2]:
import tensorflow
import pandas
import mido

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [47]:
mid = mido.MidiFile('WebsterNight.mid')

def notename(note_num):
    octave = (note_num // 12)
    notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
    note = notes[note_num % 12]
    return note + str(octave)

def ticks_to_measure(midi, ticks): 
    #print(str(midi.ticks_per_beat))
    measure_time = midi.ticks_per_beat * 4 
    return ticks / measure_time 

def dict_to_csv(dict_list, file_name): 
    l = [] 
    for dct in dict_list: 
        df = pandas.DataFrame(dct, [0])
        l.append(df)
    tmp = pandas.concat(l)
    tmp = tmp.reindex(columns = ["note_name", "start_time", "duration", "velocity"])
    tmp.to_csv(file_name + '.csv', index = False)
            
# Eventually this will all be OOP!!!
# Running variables (these will keep track of things)
def midi_to_csv(midi):
    notes = []
    curr_note = {} 
    curr_time = 0 
    for i, track in enumerate(mid.tracks):
        # print('Track {}: {}'.format(i, track.name))
        for msg in track:
            if msg.type == 'note_on': 
                curr_note['note_name'] = notename(msg.note)
                curr_note['velocity'] = msg.velocity 
                curr_time += msg.time 
                curr_note['start_time'] = ticks_to_measure(midi, curr_time)
            if msg.type == 'note_off':
                curr_note['duration'] = ticks_to_measure(midi, msg.time)
                curr_time += msg.time
                notes.append(curr_note)
                curr_note = {}    
    file_name = midi.filename[:-4]
    #print(notes)
    dict_to_csv(notes, file_name)

midi_to_csv(mid)
                